# Block 7 Übung 4: Visualisierung von Laborparametern für die Kohorten

## Bibliotheken & Konfiguration

In [ ]:
packages <- c("readr", "dplyr", "tidyr", "ggplot2")
install.packages(setdiff(packages, rownames(installed.packages())))
lapply(packages, require, character.only = TRUE)

base_url <- "https://raw.githubusercontent.com/ganslats/TMF-School-2021-Block-7/master/Rohdaten/"

## MIMIC III-Basisdaten laden

In [ ]:
# Patientenstammdaten laden
mimic.patients.raw <- read_csv(paste(base_url, "mimic-iii-demo/PATIENTS.csv", sep=""),
                               col_types = cols(row_id = col_integer(), subject_id = col_integer(), gender = col_character(), dob = col_datetime(format = ""), dod = col_datetime(format = ""), dod_hosp = col_datetime(format = ""), dod_ssn = col_datetime(format = ""), expire_flag = col_double()))

# Behandlungsfälle laden
mimic.admissions.raw <- read_csv(paste(base_url, "mimic-iii-demo/ADMISSIONS.csv", sep=""),
                                 col_types = cols(  row_id = col_integer(), subject_id = col_integer(), hadm_id = col_integer(), admittime = col_datetime(format = ""), dischtime = col_datetime(format = ""), deathtime = col_datetime(format = ""), admission_type = col_character(), admission_location = col_character(), discharge_location = col_character(), insurance = col_character(), language = col_character(), religion = col_character(), marital_status = col_character(), ethnicity = col_character(), edregtime = col_datetime(format = ""), edouttime = col_datetime(format = ""), diagnosis = col_character(), hospital_expire_flag = col_double(), has_chartevents_data = col_double()))

# Intensivaufenthalte laden
mimic.icustays.raw <- read_csv(paste(base_url, "mimic-iii-demo/ICUSTAYS.csv", sep=""),
                               col_types = cols(row_id = col_integer(), subject_id = col_integer(), hadm_id = col_integer(), icustay_id = col_integer(), dbsource = col_character(), first_careunit = col_character(), last_careunit = col_character(), first_wardid = col_double(), last_wardid = col_double(), intime = col_datetime(format = ""), outtime = col_datetime(format = ""), los = col_double()))


## MIMIC III-Diagnosen laden & Bezeichner ergänzen

In [ ]:
# Diagnosen laden
mimic.diagnoses.raw <- read_csv(paste(base_url, "mimic-iii-demo/DIAGNOSES_ICD.csv", sep=""),
                                col_types = cols(row_id = col_integer(), subject_id = col_integer(), hadm_id = col_integer(), seq_num = col_integer(), icd9_code = col_character()))

# Diagnosebezeichner laden
mimic.d_icd_diagnoses.raw <- read_csv(paste(base_url, "mimic-iii-demo/D_ICD_DIAGNOSES.csv", sep=""),
                                      col_types = cols(row_id = col_integer(), icd9_code = col_character(), short_title = col_character(), long_title = col_character()))

# Diagnosebezeichner hinzufügen
mimic.diagnoses.annotated <- mimic.diagnoses.raw %>% inner_join(mimic.d_icd_diagnoses.raw %>% select(icd9_code, short_title, long_title), by = "icd9_code")

## MIMIC III-Labordaten laden & Bezeichner ergänzen

In [ ]:
# Laborbefunde laden
mimic.labevents.raw <- read_csv(paste(base_url, "mimic-iii-demo/LABEVENTS.csv", sep=""),
                                col_types = cols(row_id = col_integer(), subject_id = col_integer(), hadm_id = col_integer(), itemid = col_integer(), charttime = col_datetime(format = ""), value = col_character(), valuenum = col_double(), valueuom = col_character(), flag = col_character()))

# Bezeichner laden
mimic.d_labitems.raw <- read_csv(paste(base_url, "mimic-iii-demo/D_LABITEMS.csv", sep=""),
                                 col_types = cols(row_id = col_integer(), itemid = col_integer(), label = col_character(), fluid = col_character(), category = col_character(), loinc_code = col_character()))

# Bezeichner hinzufügen
mimic.labevents.annotated <- mimic.labevents.raw %>% 
  inner_join(mimic.d_labitems.raw %>% select(itemid, label, fluid, category, loinc_code), by = "itemid")

head(mimic.labevents.annotated)

## Kohorten für Sepsis (sep), akuten Myokardinfarkt (ami) und akutes Nierenversagen (acf) erneut selektieren

In [ ]:
# Sepsis-Kohorte
cohort.sep <- mimic.diagnoses.annotated %>% 
    filter(grepl("Sepsis", long_title)) %>%
    select(hadm_id) %>%
    distinct() %>%
    mutate(cohort_sep_flag = TRUE)

# AMI-Kohorte
cohort.ami <- mimic.diagnoses.annotated %>% 
    filter(grepl("Acute myocardial infarction", long_title)) %>%
    select(hadm_id) %>%
    distinct() %>%
    mutate(cohort_ami_flag = TRUE)

# ACF-Kohorte
cohort.acf <- mimic.diagnoses.annotated %>% 
    filter(grepl("Acute kidney failure", long_title)) %>%
    select(hadm_id) %>%
    distinct() %>%
    mutate(cohort_acf_flag = TRUE)

## Falltabelle (admissions) um die Kohorten-Flags ergänzen

Die Aufnahmetabelle wird im folgenden Block schrittweise mit left Joins um die Flags der 3 Kohorten ergänzt. Ergebnis ist die vollständige Falltabelle mit 3 zusätzlichen Spalten, in denen jeweils mit TRUE oder FALSE angegeben ist, ob dieser Fall in eine der drei Kohorten (oder ggf. mehrere) fällt.

In [ ]:
cohort.all <- mimic.admissions.raw %>%
    left_join(cohort.sep, by = "hadm_id") %>%  # Sepsis-Kohorte hinzufügen
    left_join(cohort.ami, by = "hadm_id") %>%  # AMI-Kohorte hinzufügen
    left_join(cohort.acf, by = "hadm_id") %>%  # ACF-Kohorte hinzufügen
    replace_na(list(cohort_sep_flag = FALSE,   # "NA"-Einträge in den Kohorten-Flags durch FALSE ersetzen
                    cohort_ami_flag = FALSE,
                    cohort_acf_flag = FALSE))

head(cohort.all)

## Laboranalyte für Troponin (Herzenzym, u.a. bei Myokardinfarkt erhöht) heraussuchen

In [ ]:
mimic.d_labitems.raw %>%
    filter(grepl("Troponin", label))

### Häufigkeit der gefundenen Troponin-Analyte in den Befunden anzeigen

In [ ]:
mimic.labevents.annotated %>%
    filter(grepl("Troponin", label)) %>%
    group_by(label, fluid, loinc_code) %>%
    summarize(n = n(), .groups = "keep")

## Verteilung der Troponin-Befunde mit/ohne Zugehörigkeit zur Herzinfarkt-Kohorte visualisieren

In [ ]:
# Labordaten auf Troponin filtern & Kohortenflag ergänzen
troponin <- mimic.labevents.annotated %>%
    filter(grepl("Troponin", label)) %>%
    inner_join(cohort.all %>% select(hadm_id, cohort_ami_flag), by = "hadm_id")
head(troponin)

# Boxplot ausgeben
options(repr.plot.width = 10, repr.plot.height = 5)
ggplot(data = troponin, aes(x = label, y = valuenum, fill = cohort_ami_flag)) +
    geom_boxplot()

## Aufgabe: Finden & visualisieren Sie passende Laborbefunde für die ACF- und Sepsis-Kohorten

Hier können Sie das Wissen aus dem Abschnitt 4, Demo 2 (Daten abfragen mit Tidyverse) und dem Abschnitt 6, Demo 1 (Daten visualisieren, Diagramme) anwenden.
* zugehörige Notebooks: [teil-4-demo-02-abfragen-tidyverse.ipynb](https://github.com/ganslats/TMF-School-2021-Block-7/blob/main/Notebooks/teil-4-demo-02-abfragen-tidyverse.ipynb), [teil-6-demo-01-visualisieren-diagramme.ipynb](https://github.com/ganslats/TMF-School-2021-Block-7/blob/main/Notebooks/teil-6-demo-01-visualisieren-diagramme.ipynb)
* nutzen Sie gerne auch das [ggplot2-Cheat Sheet von RStudio](https://github.com/rstudio/cheatsheets/raw/master/data-visualization-2.1.pdf)

Sonstige Tipps:
* nutzen Sie die `category`-Spalte in der Laborbefundtabelle und achten Sie auch darauf, dass gleichnamige Untersuchungen an verschiedenen Materialien (z.B. Blut, Urin) durchgeführt werden können
* falls der Bezeichner (`label`) eines Analyts für sich nicht eindeutig in Bezug auf das Untersuchungsmaterial ist, können Sie alternativ auch  dessen LOINC-Code als Filterkriterium nutzen, da es eindeutig inkl. des verwendeten Untersuchungsmaterials ist
* suchen Sie ggf. auch nach Synonymen, da die Benennung selbst innerhalb dieses Datensatzes nicht immer einheitlich ist (z.B. *Leukocytes* und *White blood cells*)

### Visualisierung Laborbefunde für die ACF-Kohorte (akutes Nierenversagen)

In [ ]:
# Verfügbare Analyte & Befunde zum Kreatinin anzeigen
mimic.labevents.annotated %>%
    filter(grepl("Creatinine", label)) %>%
    group_by(label, fluid, loinc_code) %>%
    summarize(n = n(), .groups = "keep")

# Labordaten auf Kreatinin (im Blut) filtern & Kohortenflag ergänzen
creatinine <- mimic.labevents.annotated %>%
    filter(loinc_code == '2160-0') %>%
    inner_join(cohort.all %>% select(hadm_id, cohort_acf_flag), by = "hadm_id") %>%
    filter(valuenum < 100) # einzelnen Outlier ausfiltern
head(creatinine)

# Boxplot ausgeben
options(repr.plot.width = 10, repr.plot.height = 5)
ggplot(data = creatinine, aes(x = label, y = valuenum, fill = cohort_acf_flag)) +
    geom_boxplot()

### Visualisierung Laborbefunde für die Sepsis-Kohorte

In [ ]:
# Verfügbare Analyte & Befunde zu Leukozyten (White Blood Cells) anzeigen
mimic.labevents.annotated %>%
    filter(grepl("White Blood Cells", label)) %>%
    group_by(label, fluid, loinc_code) %>%
    summarize(n = n(), .groups = "keep")

# Labordaten auf Leukozyten filtern & Kohortenflag ergänzen
leukocytes <- mimic.labevents.annotated %>%
    filter(loinc_code == '804-5') %>%
    inner_join(cohort.all %>% select(hadm_id, cohort_sep_flag), by = "hadm_id")
head(leukocytes)

# Boxplot ausgeben
options(repr.plot.width = 10, repr.plot.height = 5)
ggplot(data = leukocytes, aes(x = label, y = valuenum, fill = cohort_sep_flag)) +
    geom_boxplot()